In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("train.csv",index_col=0)

In [3]:
from sklearn.metrics import make_scorer
sample = pd.read_csv("sample_submission.csv",index_col=0)
test = pd.read_csv("test.csv",index_col=0)
sample
def in_top_three_scoring(y_true, y_proba):
    top_3_indices = np.flip(np.argsort(y_proba, axis=1)[:, -3:], axis=1)
    accurate_counts = 0
    for i, val in enumerate(y_true):
        if val in top_3_indices[i]:
            accurate_counts += 1
    return accurate_counts / len(y_true)
top_three_scorer = lambda estimator, X, y : in_top_three_scoring(y, estimator.predict_proba(X))
def mapper(x):
    return fert_encoder.classes_[x]
vectorized_mapper = np.vectorize(mapper)
def predict_test_values(estimator):
    y_test_pred = estimator.predict_proba(test)
    y_test_pred_top3 = np.flip(np.argsort(y_test_pred, axis=1)[:, -3:], axis=1)
    y_test_pred_strings = vectorized_mapper(y_test_pred_top3)
    aaa = np.apply_along_axis(" ".join, arr=y_test_pred_strings, axis=1)
    return aaa

In [ ]:
param_grid = {
    'gb__n_estimators': [50, 100, 200],
    'gb__learning_rate': [0.01, 0.1, 0.2],
    'gb__max_depth': [3, 5, 7],
    'gb__subsample': [0.8, 1.0]
}
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
fert_encoder = LabelEncoder()
X = df.drop(columns=['Fertilizer Name'])
y=fert_encoder.fit_transform(df['Fertilizer Name'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2,random_state=42)
preprocessor = make_column_transformer((StandardScaler(),("Temparature","Humidity","Moisture","Nitrogen","Potassium","Phosphorous")),(OneHotEncoder(),("Soil Type","Crop Type")))
GB_OHE = Pipeline([('proc',preprocessor),('gb',GradientBoostingClassifier())])
grid_search_gb = RandomizedSearchCV(GB_OHE, param_grid, cv=5, scoring=top_three_scorer, n_jobs=5,verbose=2)
# Use this to test ^ before running full grid search
# grid_search_gb = GridSearchCV(GB_OHE, param_grid, cv=5, scoring=top_three_scorer, n_jobs=5,verbose=2)
grid_search_gb.fit(X,y)



Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [ ]:
grid_gb = grid_search.best_estimator_
print(grid_gb.best_params_)
print(grid_gb.best_score_)
best_score_gb = make_pipeline(('proc',preprocessor),('gb',GradientBoostingClassifier(verbose=1)))
best_score_gb.set_params(**grid_gb.best_params_)
best_score_gb.fit(X_train,y_train)
y_pred = SGD_OHE.predict_proba(X_test)
in_top_three_scoring(y_test,y_pred)
print(y_pred)
print(in_top_three_scoring(y_test,y_pred))
best_score_gb.fit(X,y)
sample['Fertilizer Name'] = predict_test_values(best_score_gb)

In [ ]:
sample.to_csv("attempt3")